In [0]:
df_raw = spark.table("ds25_wp1.consumer_bronze.consumer")

In [0]:
df_raw.filter(df_raw['Latitude'].isNotNull() & df_raw['Longitude'].isNotNull()).count()


21255641

In [0]:
selected_cols = [
    'CustomerID', 'ZIPCode', 'State', 'Latitude', 'Longitude',
    'PresenceOfChildrenInd', 'FoodWines', 'Jewelry',
    'UpscaleLiving', 'OnlinePurchasingIndicator'
]


df = (
    df_raw
    .filter(df_raw['Latitude'].isNotNull() & df_raw['Longitude'].isNotNull())
    .select(*selected_cols)
    .sample(False, 0.01, seed=42)
    .toPandas()
)


In [0]:
df.shape

(212536, 10)

In [0]:
binary_cols = [
    'PresenceOfChildrenInd',
    'FoodWines',
    'Jewelry',
    'UpscaleLiving',
    'OnlinePurchasingIndicator'
]

for col in binary_cols:
    df[col] = df[col].map({'Y': 1, 'N': 0})



In [0]:
def is_ideal(row):
    return (
        row['PresenceOfChildrenInd'] == 1 and
        row['FoodWines'] == 1 and
        row['UpscaleLiving'] == 1
    )

df['IdealCustomer'] = df.apply(is_ideal, axis=1)


In [0]:
import pandas as pd
zip_summary = (
    df[df['IdealCustomer'] == True]
    .groupby('ZIPCode')
    .agg(
        ideal_count=('CustomerID', 'count'),
        Latitude=('Latitude', 'first'),
        Longitude=('Longitude', 'first')
    )
    .sort_values('ideal_count', ascending=False)
    .reset_index()
)

# STEP 2: Clean after creation
zip_summary['Latitude'] = pd.to_numeric(zip_summary['Latitude'], errors='coerce')
zip_summary['Longitude'] = pd.to_numeric(zip_summary['Longitude'], errors='coerce')
zip_summary = zip_summary.dropna(subset=['Latitude', 'Longitude'])



In [0]:
zip_summary.shape


(626, 4)

In [0]:
df['IdealCustomer'].value_counts(dropna=False)

False    210845
True       1691
Name: IdealCustomer, dtype: int64

In [0]:
df[['Latitude', 'Longitude']].dropna().head()

,Latitude,Longitude
0,30.177795,-81.753150
1,28.918529,-81.945848
2,28.921654,-81.947432
3,30.279232,-83.030387
4,30.567437,-83.147350


In [0]:
import plotly.express as px
import requests
import json

# Load Florida GeoJSON directly from GitHub
url = "https://raw.githubusercontent.com/glynnbird/usstatesgeojson/master/florida.geojson"
response = requests.get(url)
florida_geojson = response.json()

# Create the scatter map
fig = px.scatter_mapbox(
    zip_summary,
    lat="Latitude",
    lon="Longitude",
    size="ideal_count",
    color="ideal_count",
    hover_name="ZIPCode",
    mapbox_style="carto-darkmatter",  # Dark style for contrast (optional)
    zoom=6,
    center={"lat": 27.8, "lon": -82.5}
)

# Add Florida boundary overlay
fig.update_layout(
    mapbox={
        "layers": [{
            "source": florida_geojson,
            "type": "line",
            "color": "white",
            "line": {"width": 2}
        }],
        "bounds": {
            "west": -88.0,   # Westernmost part of Florida Panhandle
            "east": -79.5,   # Easternmost tip near Miami
            "south": 24.3,   # Southern Key West
            "north": 31.0    # Northern edge bordering Georgia
        }
    },
    margin={"r":0, "t":0, "l":0, "b":0}
)

fig.show()

"""
fig = px.scatter_mapbox(
    zip_summary,
    lat="Latitude",
    lon="Longitude",
    size="ideal_count",
    color="ideal_count",
    hover_name="ZIPCode",
    mapbox_style="carto-positron",
    zoom=5
)

fig.show() """


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-6002457635941688>, line 24
     11 fig = px.scatter_mapbox(
     12     zip_summary,
     13     lat="Latitude",
   (...)
     20     center={"lat": 27.8, "lon": -82.5}
     21 )
     23 # Add Florida boundary overlay
---> 24 fig.update_layout(
     25     mapbox={
     26         "layers": [{
     27             "source": florida_geojson,
     28             "type": "line",
     29             "color": "white",
     30             "line": {"width": 2}
     31         }],
     32         "bounds": {
     33             "west": -88.0,   # Westernmost part of Florida Panhandle
     34             "east": -79.5,   # Easternmost tip near Miami
     35             "south": 24.3,   # Southern Key West
     36             "north": 31.0    # Northern edge bordering Georgia
     37         }
     38     },
     39     margin={"r":0

In [0]:
import pandas as pd
from io import StringIO

# Convert DataFrame to CSV string
csv_data = StringIO()
df.to_csv(csv_data, index=False)
csv_data.seek(0)

# Write CSV string to DBFS
dbutils.fs.put("/FileStore/cleaned_customers.csv", csv_data.getvalue(), overwrite=True)

# Repeat for the second DataFrame
csv_data = StringIO()
zip_summary.to_csv(csv_data, index=False)
csv_data.seek(0)

dbutils.fs.put("/FileStore/zip_summary.csv", csv_data.getvalue(), overwrite=True)

Wrote 16206672 bytes.
Wrote 18061 bytes.


True

In [0]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/731.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/6.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 129.0 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
    Not uninstalling blinker at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-ddef3110-a1f1-4ca1-8f85-7432ea75bbe3
    Can't uninstall 'blinker'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Load your cleaned dataset from DBFS using Spark
df_spark = spark.read.csv("/FileStore/cleaned_customers.csv", header=True, inferSchema=True)
df = df_spark.toPandas()

# Title
st.title("Find Your Ideal Customers by ZIP Code")

# Sidebar filters
st.sidebar.header("Define Your Ideal Customer")
traits = {
    'Has Children': 'PresenceOfChildrenInd',
    'Food & Wine Interest': 'FoodWines',
    'Upscale Lifestyle': 'UpscaleLiving',
    'Online Purchaser': 'OnlinePurchasingIndicator',
    'Likes Jewelry': 'Jewelry'
}

selected_traits = [v for k, v in traits.items() if st.sidebar.checkbox(k)]

# Only run the rest if user selected at least one checkbox
if selected_traits:
    # Ensure selected columns are numeric
    for col in selected_traits:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Filter based on number of matching traits
    filter_condition = df[selected_traits].sum(axis=1) >= len(selected_traits)
    filtered_df = df[filter_condition]

    # Group by ZIP
    zip_summary = (
        filtered_df.groupby("ZIPCode")
        .agg(
            ideal_count=('CustomerID', 'count'),
            Latitude=('Latitude', 'first'),
            Longitude=('Longitude', 'first')
        )
        .reset_index()
        .dropna(subset=['Latitude', 'Longitude'])
    )

    # Convert to numeric if needed
    zip_summary['Latitude'] = pd.to_numeric(zip_summary['Latitude'], errors='coerce')
    zip_summary['Longitude'] = pd.to_numeric(zip_summary['Longitude'], errors='coerce')

    # Map
    st.subheader("Matching ZIP Codes")
    fig = px.scatter_mapbox(
        zip_summary,
        lat="Latitude",
        lon="Longitude",
        size="ideal_count",
        color="ideal_count",
        hover_name="ZIPCode",
        mapbox_style="carto-positron",
        zoom=5
    )
    st.plotly_chart(fig)

    # Table
    st.subheader("Summary Table")
    st.dataframe(zip_summary.sort_values("ideal_count", ascending=False))

else:
    st.warning("Please select at least one trait to search.")


2025-05-08 13:47:09.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 13:47:09.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar